In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Create possible haplotypes for RN and associated colour palettes

#4 gene haplotype order and palette
firsta = ['S', 's']
seconda = ['A', 'a']
thirda = ['B', 'b']
fourtha = ['R', 'r']
hap_4gene_poss =[]

for a in firsta:
    for b in seconda:
        for c in thirda:
            for d in fourtha:
                hap_4gene_poss.append(a+b+c+d)
                
order = [0, 4, 3, 10, 2, 9, 8, 14, 1, 7, 6, 12, 5, 13, 11, 15]
hap_4gene_order = dict(zip(hap_4gene_poss, order))
 
num_cols = len(hap_4gene_poss)
cols = sns.color_palette("husl", num_cols)
col_4gene = dict(zip(hap_4gene_poss, cols))
hap_4_col = ['RN_RUNX1_g', 'RN_SRSF2', 'RN_TET2a', 'RN_TET2b_g']

#3 gene haplotype order and palette
firsta = ['S', 's']
seconda = ['A', 'a']
fourtha = ['R', 'r']
hap_3gene_poss =[] 

for a in firsta:
    for b in seconda:
        for d in fourtha:
            hap_3gene_poss.append(a+b+d)
                
order = [0, 4, 3, 10, 2, 9, 8, 14, 1, 7, 6, 12, 5, 13, 11, 15]
hap_3gene_order = dict(zip(hap_3gene_poss, order))

num_cols = len(hap_3gene_poss)
cols = sns.color_palette("husl", num_cols)
col_3gene = dict(zip(hap_3gene_poss, cols))
hap_3_col = [ 'RN_RUNX1_g', 'RN_SRSF2', 'RN_TET2a']

#2 gene haplotype order and palette
firsta = ['C', 'c']
seconda = ['G', 'g']
hap_2gene_poss =[]

for a in firsta:
    for b in seconda:
        hap_2gene_poss.append(a+b)
                
order = [0, 1, 2, 3]
hap_2gene_order = dict(zip(hap_2gene_poss, order))
            
num_cols = len(hap_2gene_poss)
cols = sns.color_palette("husl", num_cols)
col_2gene = dict(zip(hap_2gene_poss, cols))
hap_2_col = ['RN_RUNX1_c', 'RN_RUNX1_g']

In [22]:
#Import the data and flip into a multi index 

def data_retrieval(sourcefile, pt_initials):
    df = pd.read_csv(sourcefile, header = [0,1,2], index_col = 0, sep='\t')
    df = df.stack([0,1,2])
    df = df.reorder_levels([1,0,2,3])
    df = df.to_frame()  #puts everything back in a dataframe
    df.columns = ['Reads']
    df['Plate'] = df.index.get_level_values(0)  #These lines send indexes to columns
    df['Well'] = df.index.get_level_values(1)
    df['Amplicon'] = df.index.get_level_values(2)
    df['Genotype'] = df.index.get_level_values(3)
    df[['Patient', 'one', 'two']] = df['Amplicon'].str.split('_', expand = True)
    df = df.drop(columns = ['one', 'two'])

    #Import information about plate cell type and patient
    key = pd.read_excel('../Data/Amp_data/Amplicon_metadata_fixed.xlsx', sheet_name = 'PlateID') #should this be an input? also in next fucntion
    key = key.drop(['Cell Origin', 'Plate Nr', 'Plate Name','Nr of cells', 'fcs-fle' ], axis=1)
    key.rename(columns = {'Comments2':'Plate'}, inplace = True)
    key.rename(columns = {'Cell-group':'Celltype'}, inplace = True)
    
    #Make a dictionary to associate plates with patients and plate with cell type
    plate_pt_dict = dict(zip(key.Plate, key.Patient))
    plate_cell_dict = dict(zip(key.Plate, key.Celltype))

    #Now just look at data from selected patient, and apply filters to identify cells with enough reads/amplicon
    #RN_allele_plate is the key dataset going forward
    pt_allele_plate = df.loc[df['Patient'].isin([pt_initials])] #Make df with just RN data
    pt_allele_plate = pt_allele_plate.drop(columns = 'Patient') #Drop the Patient ID column and other unwanted cols
    pt_allele_plate['Cell_type'] = pt_allele_plate['Plate'].replace(plate_cell_dict)
    pt_allele_plate['Plate_Well'] = pt_allele_plate['Plate'].astype(str) + '_' + pt_allele_plate['Well'].astype(str)
    #pt_allele_plate = pt_allele_plate.groupby(['Plate', 'Well', 'Amplicon']).sum().unstack()
    
    return pt_allele_plate

def call_haps (data, haps, reads, cutoff):

#data is the df created by data_retrieval
#haps is the number of haplotypes
#reads is the number of reads per amplicon
#cutoff is the threshold for calling an amplicon as mutant
    
    #Import information about plate cell type and patient
    key = pd.read_excel('../Data/Amp_data/Amplicon_metadata_fixed.xlsx', sheet_name = 'PlateID')
    key = key.drop(['Cell Origin', 'Plate Nr', 'Plate Name','Nr of cells', 'fcs-fle' ], axis=1)
    key.rename(columns = {'Comments2':'Plate'}, inplace = True)
    key.rename(columns = {'Cell-group':'Celltype'}, inplace = True)
    
    #Make a dictionary to associate plates with patients and plate with cell type
    plate_pt_dict = dict(zip(key.Plate, key.Patient))
    plate_cell_dict = dict(zip(key.Plate, key.Celltype))
    
    #Create possible haplotypes for RN and associated colour palettes

    #4 gene haplotype order and palette
    firsta = ['S', 's']
    seconda = ['A', 'a']
    thirda = ['B', 'b']
    fourtha = ['R', 'r']
    hap_4gene_poss =[]

    for a in firsta:
        for b in seconda:
            for c in thirda:
                for d in fourtha:
                    hap_4gene_poss.append(a+b+c+d)

    order = [0, 4, 3, 10, 2, 9, 8, 14, 1, 7, 6, 12, 5, 13, 11, 15]
    hap_4gene_order = dict(zip(hap_4gene_poss, order))

    num_cols = len(hap_4gene_poss)
    cols = sns.color_palette("husl", num_cols)
    col_4gene = dict(zip(hap_4gene_poss, cols))
    hap_4_col = ['RN_RUNX1_g', 'RN_SRSF2', 'RN_TET2a', 'RN_TET2b_g']

    #3 gene haplotype order and palette
    firsta = ['S', 's']
    seconda = ['A', 'a']
    fourtha = ['R', 'r']
    hap_3gene_poss =[] 

    for a in firsta:
        for b in seconda:
            for d in fourtha:
                hap_3gene_poss.append(a+b+d)

    order = [0, 4, 3, 10, 2, 9, 8, 14, 1, 7, 6, 12, 5, 13, 11, 15]
    hap_3gene_order = dict(zip(hap_3gene_poss, order))

    num_cols = len(hap_3gene_poss)
    cols = sns.color_palette("husl", num_cols)
    col_3gene = dict(zip(hap_3gene_poss, cols))
    hap_3_col = [ 'RN_RUNX1_g', 'RN_SRSF2', 'RN_TET2a']

    #2 gene haplotype order and palette
    firsta = ['C', 'c']
    seconda = ['G', 'g']
    hap_2gene_poss =[]

    for a in firsta:
        for b in seconda:
            hap_2gene_poss.append(a+b)

    order = [0, 1, 2, 3]
    hap_2gene_order = dict(zip(hap_2gene_poss, order))

    num_cols = len(hap_2gene_poss)
    cols = sns.color_palette("husl", num_cols)
    col_2gene = dict(zip(hap_2gene_poss, cols))
    hap_2_col = ['RN_RUNX1_c', 'RN_RUNX1_g']
    
    if haps == 4:
        cols = hap_4_col
    
    elif haps == 2:
        cols = hap_2_col
        
    elif haps == 3:
        cols = hap_3_col
        
    else:
        print('Enter 2, 3, or 4 haplotypes')
    
    #Group the data and apply filters
    df = data.copy()
    df = df.groupby(['Plate', 'Well', 'Amplicon']).sum().unstack()
    df.columns = ['RN_RUNX1_c','RN_RUNX1_g','RN_SRSF2','RN_TET2a','RN_TET2b_c','RN_TET2b_g']
    
    df = df.loc[(df[cols] >= reads).all(axis=1)] #df1 contains just the rows with cells we want - use this to create a filter or key
    df['Plate'] = df.index.get_level_values(0)  #These lines send indexes to columns
    df['Well'] = df.index.get_level_values(1)
    df['Plate_Well'] = df['Plate'].astype(str) + '_' + df['Well'].astype(str)
    wells = df['Plate_Well'].drop_duplicates().to_list() 
    print(f'Cells with {reads} reads for {haps} genes = ', len(wells))
    
    df2 = data.copy()
    df2 = df2[df2['Plate_Well'].isin(wells)]
    df2 = df2[df2['Amplicon'].isin(cols)]
    
    #Calculate the allele frequency
    df2 = df2.iloc[:, 0:1].unstack(level = 3)
    df2['Total'] = df2.iloc[: , 0] + df2.iloc[: , 1]
    df2['Mut_freq'] = df2.iloc[:, 0]/df2['Total']

    #Assign Wt or MT to each allele
    df2 = df2.drop(columns = ['Reads', 'Total'])

    conditions = [(df2['Mut_freq'] <= cutoff), (df2['Mut_freq']) > cutoff ]
    values = ['w', 'm']
    df2['Genotype'] = np.select(conditions, values)
    df2 = df2.drop(columns = ['Mut_freq']).unstack(2)
    df2.columns = cols
    
    if 'RN_RUNX1_g' in df2.columns:
        df2.loc[:,'RN_RUNX1_g'].replace({'w':'R','m':'r' }, inplace = True)
        
    if 'RN_SRSF2' in df2.columns:  
        df2.loc[:,'RN_SRSF2'].replace({'w':'S','m':'s' }, inplace = True)
        
    if 'RN_TET2a' in df2.columns:     
        df2.loc[:,'RN_TET2a'].replace({'w':'A','m':'a' }, inplace = True)
        
    if 'RN_TET2b_g' in df2.columns:
        df2.loc[:,'RN_TET2b_g'].replace({'w':'B','m':'b' }, inplace = True)
        
    if 'RN_RUNX1_c' in df2.columns:   
        df2.loc[:,'RN_RUNX1_c'].replace({'w':'C','m':'c' }, inplace = True)
        
    df2['Haplotype'] = 'x'

    for idx, row in df2.iterrows():
        
        if haps == 3:
            a = row['RN_SRSF2'] + row['RN_TET2a'] + row['RN_RUNX1_g']
        elif haps == 4:
            a = row['RN_SRSF2'] + row['RN_TET2a'] + row['RN_TET2b_g'] + row['RN_RUNX1_g']
        elif haps ==2:
            a = row['RN_RUNX1_c'] + row['RN_RUNX1_g']
        
        row['Haplotype'] = row['Haplotype'].replace('x', a)   
    
    df2['Sort_cell_type'] = df2.index.get_level_values(0)
    df2['Sort_cell_type'] = df2['Sort_cell_type'].replace(plate_cell_dict)
    df2['Plate'] = df2.index.get_level_values(0)
    df2['Well'] = df2.index.get_level_values(1)
    df2['Plate_Well'] = df2['Plate'].astype(str) + '_' + df2['Well'].astype(str)
    df2 = df2.drop(columns = cols)
    df2 = df2.drop(columns = ['Plate', 'Well'])

    return df2

In [4]:
#QC just checking the output is the same.
#The columns (L-R) are; [('Reads', 'RN_RUNX1_c'), ('Reads', 'RN_RUNX1_g'), ('Reads', 'RN_SRSF2'), ('Reads', 'RN_TET2a'), ('Reads', 'RN_TET2b_c'), ('Reads', 'RN_TET2b_g')]
#Cells with 100 reads for 3 genes (plate1) =  794
#Cells with 15 reads for 4 genes (plate2) =  162
#Cells with 50 reads for RUNX1 cDNA and gDNA (plate3) =  1645
#Cells with 10 reads for 4 genes (plate4) =  253
#Cells with 50 reads for 3 genes (plate5) =  1234

In [25]:
datafile = '../Data/Amp_data/allele_counts.tsv'
index_data = pd.read_csv('../Data/Amp_data/RN_comp_celltype_assignment.tsv', index_col = 0, sep = '\t')
RN_plate = data_retrieval(datafile, 'RN')  
RN_haps = call_haps(RN_plate, 3, 100, 0.1) #hap number, read cut, proprtion mutated
RN_haps_indexed = pd.merge(RN_haps, index_data, on = 'Plate_Well')
RN_haps_indexed

Cells with 100 reads for 3 genes =  794


,Haplotype,Sort_cell_type,Plate_Well,FSC-A,FSC-W,FSC-H,SSC-A,SSC-W,SSC-H,CD34-PE,...,CD66b_pos,CD66b_neg,CD16_pos,CD16_neg,CD14_pos,CD14_neg,nBC,NE,CD14pos_mono,CD14neg_mono
0,SAR,nBCs,AS-194_A12,78240.600,85805.690,59758.0,57048.51,77708.920,48112.0,NaN,...,False,True,False,True,False,True,True,False,False,False
1,SAR,nBCs,AS-194_A13,83133.000,91049.234,59838.0,23547.42,82927.820,18609.0,NaN,...,False,True,False,True,False,True,True,False,False,False
2,SAR,nBCs,AS-194_A16,70706.695,87015.450,53253.0,28536.00,76133.170,24564.0,NaN,...,False,True,False,True,False,True,True,False,False,False
3,SaR,nBCs,AS-194_A22,50627.700,82859.350,40043.0,17333.88,79512.375,14287.0,NaN,...,False,True,False,True,False,True,True,False,False,False
4,SAR,nBCs,AS-194_B13,70561.800,80219.586,57646.0,13657.26,70497.960,12696.0,NaN,...,False,True,False,True,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,SaR,NEs,AS-195_O22,131130.000,99892.310,86030.0,127044.36,90787.920,91708.0,NaN,...,True,False,True,False,False,True,False,True,False,False
788,SAR,NEs,AS-195_P18,128855.695,105181.260,80287.0,118534.02,94177.680,82485.0,NaN,...,True,False,True,False,False,True,False,True,False,False
789,saR,NEs,AS-189_C2,115577.090,102051.414,74222.0,118898.55,91965.380,84729.0,NaN,...,True,False,True,False,False,True,False,True,False,False
790,saR,NEs,AS-189_D9,148525.200,110377.480,88186.0,141337.60,98506.890,94031.0,NaN,...,True,False,True,False,False,True,False,True,False,False


In [26]:
def plot_hap_dist_sort_type(data):
    
    #4 gene haplotype order and palette
    firsta = ['S', 's']
    seconda = ['A', 'a']
    thirda = ['B', 'b']
    fourtha = ['R', 'r']
    hap_4gene_poss =[]

    for a in firsta:
        for b in seconda:
            for c in thirda:
                for d in fourtha:
                    hap_4gene_poss.append(a+b+c+d)

    order = [0, 4, 3, 10, 2, 9, 8, 14, 1, 7, 6, 12, 5, 13, 11, 15]
    hap_4gene_order = dict(zip(hap_4gene_poss, order))

    num_cols = len(hap_4gene_poss)
    cols = sns.color_palette("husl", num_cols)
    col_4gene = dict(zip(hap_4gene_poss, cols))
    hap_4_col = ['RN_RUNX1_g', 'RN_SRSF2', 'RN_TET2a', 'RN_TET2b_g']

    #3 gene haplotype order and palette
    firsta = ['S', 's']
    seconda = ['A', 'a']
    fourtha = ['R', 'r']
    hap_3gene_poss =[] 

    for a in firsta:
        for b in seconda:
            for d in fourtha:
                hap_3gene_poss.append(a+b+d)

    order = [0, 4, 3, 10, 2, 9, 8, 14, 1, 7, 6, 12, 5, 13, 11, 15]
    hap_3gene_order = dict(zip(hap_3gene_poss, order))

    num_cols = len(hap_3gene_poss)
    cols = sns.color_palette("husl", num_cols)
    col_3gene = dict(zip(hap_3gene_poss, cols))
    hap_3_col = [ 'RN_RUNX1_g', 'RN_SRSF2', 'RN_TET2a']

    #2 gene haplotype order and palette
    firsta = ['C', 'c']
    seconda = ['G', 'g']
    hap_2gene_poss =[]

    for a in firsta:
        for b in seconda:
            hap_2gene_poss.append(a+b)

    order = [0, 1, 2, 3]
    hap_2gene_order = dict(zip(hap_2gene_poss, order))

    num_cols = len(hap_2gene_poss)
    cols = sns.color_palette("husl", num_cols)
    col_2gene = dict(zip(hap_2gene_poss, cols))
    hap_2_col = ['RN_RUNX1_c', 'RN_RUNX1_g']
    
    #rename the input data and work out how many haplotypes it has
    df3 = data.copy()
    haps = len(df3.iloc[0,0])
    print(a)

In [27]:
plot_hap_dist_sort_type(RN_haps)

#Ready to add in plotting info, need to work out whihc plot to use (depending on )

c


In [12]:
index_data = pd.read_csv('../Data/Amp_data/RN_comp_celltype_assignment.tsv', sep = '/t')

<ipython-input-12-c51401a78f3d>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  index_data = pd.read_csv('../Data/Amp_data/RN_comp_celltype_assignment.tsv', sep = '/t')
